In [3]:
%cd /content/drive/MyDrive/CS410

/content/drive/MyDrive/CS410


In [5]:
!pip install celluloid

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import numpy as np
import math
import pandas as pd
from IPython import display
from time import sleep
import math
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from celluloid import Camera

In [7]:
def Sphere(ind):
    sum = 0
    for i in ind:
        sum += i**2
    return sum

In [8]:
def Zakharov(x):
  d = len(x)
  sum1 = 0.0
  sum2 = 0.0
  for i in range(d):
    sum1 += x[i]**2
    sum2 += 0.5 * (i + 1) * x[i]
  y = sum1 + sum2**2 + sum2**4
  return y

In [9]:
def Rosenbrock(ind):
    sum = 0
    for i in range(len(ind) - 1):
        sum += 100 * (ind[i + 1] - ind[i]**2)**2 + (ind[i] - 1)**2
    return sum 

In [10]:
def Michalewicz(x):
  d = len(x)
  sum = 0.0
  for i in range(d):
    sum += np.sin(x[i])*pow((np.sin((i+1)*pow(x[i],2)/np.pi)),20) 
  return sum

In [11]:
def Ackley(d):
    a = 20
    b = 0.2
    c = 2 * np.pi
    sum1 = 0
    sum2 = 0
    for i in range(len(d)):
        sum1 += d[i] ** 2
        sum2 += np.cos(c * d[i])
    term1 = -a * np.exp(-b * np.sqrt(sum1 / len(d)))
    term2 = -np.exp(sum2 / len(d))

    return term1 + term2 + a + np.exp(1)

In [12]:
def DE(test_function, dimension, bounds, F_scale, cross_prob, popsize, max_evals):

    eps = 0.0001

    bound_lower, bound_upper = np.asarray(bounds).T

    diff = np.fabs(bound_lower - bound_upper)

    pop = bound_lower + diff * np.random.rand(popsize, dimension)

    fitness = np.asarray([test_function(ind) for ind in pop])
    num_eval = 1
    
    best_idx = np.argmin(fitness)
    best = pop[best_idx]

    results = []
    all_pops = []
    results.append((np.copy(best), fitness[best_idx], num_eval))
    all_pops.append(np.copy(pop))
    generation_count = 0
    
    while True:
        if num_eval > max_evals:
            break
        for i in range(popsize):
            # Mutation step
            idxes = [idx for idx in range(popsize) if idx != i]
            a, b, c = pop[np.random.choice(idxes, 3, replace=False)]
            mutant = np.clip(F_scale*(b - c) + a, bound_lower, bound_upper)

            # Create cross point
            cross_points = np.random.rand(dimension) < cross_prob
            if not np.any(cross_points):
                cross_points[np.random.randint(0, dimension)] = True
            
            # Offspring
            trial = np.where(cross_points, mutant, pop[i])

            # Evaluate fitness
            f = test_function(trial)
            num_eval += 1

            if f < fitness[i]:
                pop[i] = trial
                fitness[i] = f 
                if f < fitness[best_idx]:
                    best = trial
                    best_idx = i

        results.append((np.copy(best), fitness[best_idx], num_eval))
        all_pops.append(np.copy(pop))

        if test_function(best) < eps:
            num_eval += 1
            break

        generation_count += 1

    return results, all_pops, generation_count


In [16]:
seed_number = 20521394

bound_lower, bound_upper = -5, 5
popsize = 32
dimension = 2
max_evals = 1e5

F_scale=0.8
cross_prob=0.7

In [17]:
def DifferentialEvolution(test_function, dimension, bound_lower, bound_upper, F_scale, cross_prob, popsize, max_evals):
    np.random.rand(seed_number)
    
    results, all_pops, generation_count = DE(test_function, dimension, [(bound_lower, bound_upper)]*dimension, F_scale, cross_prob, popsize, max_evals)
    bound_lower = -30
    bound_upper = 30
    x = np.linspace(bound_lower, bound_upper, 100)
    y = np.linspace(bound_lower, bound_upper, 100)
    X, Y = np.meshgrid(x, y)
    Z = test_function([X, Y])
    
    
    fig = plt.figure(figsize=(12, 12))
    camera = Camera(fig)
    plt.contourf(X, Y, Z, popsize, cmap='viridis')
    plt.axis('square')
    plt.scatter(0, 0, marker='*')
    
    for generation in range(generation_count):
        plt.contourf(X, Y, Z, popsize, cmap='viridis')
        plt.axis('square')
        plt.scatter(0, 0, marker='*')
        plt.scatter(all_pops[generation][:, 0], all_pops[generation][:, 1], c='red', marker='o')
        plt.plot()
        plt.pause(0.01)
        camera.snap()
    plt.show()
    anim = camera.animate()
    #save the animation as a gif file
    anim.save('Michalewicz' + "-DE-" + str(popsize) + ".gif",writer="pillow")

In [19]:
all_fitness = []
num_evaluation = []
    
test_function = Ackley
seed_number = 20521394
    
popsize_array = [32, 64, 128, 256, 512, 1024]
for popsize in popsize_array:
    print(f"Popsize = {popsize}")
    if dimension == 2:
        max_evals = 1e5
        bounds = [(bound_lower, bound_upper)]*dimension
        mean_result = np.mean(all_fitness)
        stddev_result = np.std(all_fitness)
        mean_evaluation = np.mean(num_evaluation)
            
        DifferentialEvolution(test_function, dimension, bound_lower, bound_upper, F_scale, cross_prob, popsize, max_evals)

Output hidden; open in https://colab.research.google.com to view.